In [ ]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install celery dcdatabase multiprocess

In [ ]:
from celery import Celery
from kombu import Exchange, Queue
import requests
from json import loads

# Common definitions used through the different code blocks. Do not check in secrets.

# Abuse API username/password
username = ''
password = ''
# AMQP broker connection string.
broker_url = ''
# Mongo Connection string
mongo_url = ''

def get_jwt():
    response = requests.post(f'https://sso.godaddy.com/v1/api/token', json={'username': username, 'password': password}, params={'realm': 'idp'})
    response.raise_for_status()
    body = loads(response.text)
    return body.get('data')

class CeleryConfig:
    # Celery settings
    broker_transport = 'pyamqp'
    broker_use_ssl = True
    worker_hijack_root_logger = False
    result_backend = 'rpc://'
    result_serializer = 'pickle'
    accept_content = ['json', 'pickle']
    WORKER_ENABLE_REMOTE_CONTROL = True
    result_expires = 3600

    broker_url = broker_url
    queue_args = {'x-queue-type': 'quorum'}
    zeus_queue = Queue('zeus', Exchange('zeus'), routing_key='zeus', queue_arguments=queue_args)
    gdbs_queue = Queue('gdbrandservice', Exchange('gdbrandservice'), routing_key='gdbrandservice', queue_arguments=queue_args)

    task_routes = {
        'run.soft_intentionally_malicious': {'queue': zeus_queue},
        'run.add_note': {'queue': gdbs_queue},
    }


def get_celery() -> Celery:
    capp = Celery()
    capp.config_from_object(CeleryConfig)
    return capp

class Settings:
    DBURL = mongo_url
    DB = 'phishstory'
    COLLECTION = 'incidents'

In [ ]:
import requests
from functools import partial
from json import loads

# Read URLs from `sites.txt`(formated one site per line, Nix line endings) and create
# Abuse API tickets for them, saving the ticket ID to `tickets.txt`. The rate here
# needs to be monitored carefully to ensure we don't overwhelm CMAP service or back
# up other event processing. Should update the info note to reflect your use case.

jwt = get_jwt()

def create_incident(site):
    incident = {
        'type': 'PHISHING',
        'source': f'https://{site}',
        'proxy': 'skip',
        'reporter': 'jomax\\nwade',
        'info': 'large gocentral suspension task to cleanup clickto link spam'
    }
    global jwt
    headers = {'Authorization': jwt}
    error_string = ''
    try:
        api_call = partial(requests.post, 'https://abuse.api.int.godaddy.com/v1/abuse/tickets', json=incident, headers=headers)
        r = api_call()
        if r.status_code in [401, 403]:
            jwt = get_jwt()
            headers = {'Authorization': jwt}
            r = api_call()

        if r.status_code == 201:
            return True, r.text
        else:
            error_string = r.text
    except Exception as e:
        print(f'Exception while creating ticket {incident} {e}')
        error_string = str(e)
    return False, error_string

sites = []
with open('sites.txt', 'r') as f:
    with open('tickets.txt', 'a') as o:
        for x in f:
            x = x.strip()
            status, json_data = create_incident(x)
            if status:
                data = loads(json_data)
                o.write(data['u_number'] + '\n')
                o.flush()
            else:
                print(x)
                raise Exception(f'Failed on {x}')


In [ ]:
import requests
from functools import partial
import multiprocess as mp

# Read tickets from tickets.txt and close them with the close reason,
# This script runs multiple threads so for large ticket quantities
# you will need monitor to ensure events don't backup.

jwt = get_jwt()

def close_incident(ticket):
    payload = {
        'closed': 'true',
        'close_reason': 'soft_intentionally_malicious'
    }
    global jwt
    headers = {'Authorization': jwt}
    error_string = ''
    try:
        api_call = partial(requests.patch, f'https://abuse.api.int.godaddy.com/v1/abuse/tickets/{ticket}', json=payload, headers=headers)
        r = api_call()
        if r.status_code in [401, 403]:
            jwt = get_jwt()
            headers = {'Authorization': jwt}
            r = api_call()

        if r.status_code == 204:
            print(f'Closed {ticket}')
            return True
    except Exception as e:
        print(f'Exception while creating ticket {ticket} {e}')
        error_string = str(e)
    return False, error_string

with open('tickets.txt', 'r') as f:
    tickets = []
    for x in f:
        tickets.append(x.strip())

    with mp.Pool(10) as p:
        p.map(close_incident, tickets)


In [ ]:
celery = None

# Read tickets from tickets.txt and run the Zeus task and add a note via GDBS.

def send_celery_tasks(ticket):
    global celery
    if not celery:
        celery = get_celery()
    celery.send_task('run.soft_intentionally_malicious', args=(ticket, 'nwade',))
    celery.send_task('run.add_note', args=('Account suspended as it is thought to be intentionally malicious, however a soft int mal was used.', 'automation', None, ticket,))


with open('tickets.txt', 'r') as f:
    tickets = []
    for x in f:
        tickets.append(x.strip())

    with mp.Pool(10) as p:
        p.map(send_celery_tasks, tickets)

In [ ]:
from dcdatabase.phishstorymongo import PhishstoryMongo
import datetime

# Read tickets from tickets.txt and add the actions to the log.

db = None

def update_mongo_fields(ticket):
    global db
    if not db:
        db = PhishstoryMongo(Settings())
    now = datetime.datetime.utcnow()
    db._mongo.update_incident(ticket, dict(last_modified=now), unset=False, upsert=True)
    db.update_actions_sub_document(ticket, 'email sent hosted_shopper_suspend_intentional_notice', 'automation')
    db.update_actions_sub_document(ticket, 'closed as soft_intentionally_malicious', 'automation')

with open('tickets.txt', 'r') as f:
    tickets = []
    for x in f:
        tickets.append(x.strip())

    with mp.Pool(10) as p:
        p.map(update_mongo_fields, tickets)